In [1]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

## Load training dataset

In [2]:
# Root
from CustomDataset import CustomDataset

root = '../../train_val_test_dataset/IEEE_Case1354/agg_PG/'

# Load all data with empty input
dataset = CustomDataset(root=root, data_list=[])

# Set train and test size
train_size = 1000
test_size = 200

# Set batch size
batch_size = 10

# Get train dataset
train_dataset = dataset[:train_size]
# Create train loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Set hyper-parameters
input_dim = 24
hidden_dim = 256
output_dim = 12

# Set device
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## SAGE

In [ ]:
from GNNAggPGRegressor import AggPGSAGE

# Initialize the model
model = AggPGSAGE(input_dim, hidden_dim, output_dim)
model.to(device)

#### SAGE model training

In [ ]:
epochs = 1000

# Set optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Set model to training mode
model.train()

# Model training and validation
epoch_loss = []
start_time = time.time()
for epoch in range(epochs):
    batch_loss = []
    for loader in train_loader:
        loader.to(device)

        optimizer.zero_grad()

        y_pred = model(loader.x, loader.edge_index, loader.batch)

        loss = F.mse_loss(y_pred, loader.y)

        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
    epoch_loss.append(np.mean(batch_loss))

end_time = time.time()
training_time = (end_time - start_time)/60

plt.figure(figsize=(5, 3))
plt.plot(epoch_loss)
plt.yscale('log')
plt.xlabel('Epoch')
plt.ylabel('MSE loss')
plt.title(f'Training time: {training_time:.2f}')
plt.show()
plt.close('all')

#### Save SAGE model

In [5]:
torch.save(model, f'../../trained_model/IEEE_Case1354/agg_PG_model.pt')

#### SAGE model testing

In [6]:
unit_length = 9
model_path = f'../../trained_model/IEEE_Case1354/agg_PG_model.pt'
model = torch.load(model_path)
model.to('cpu')

model.eval()

# Complete dataset
test_dataset = dataset[-test_size:]
############ Shuffle cannot be true for testing data ##############
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

with torch.no_grad():
    for i, loader in enumerate(test_loader):
        # Get ground true
        ground_true = loader.y.cpu().detach().numpy()
        # Save ground true
        for j in range(batch_size):
            file_path = f'../../res/IEEE_Case1354/model_evaluation/agg_PG/true/true_{i*batch_size+j+1}.csv'
            pd.DataFrame(ground_true[j*unit_length:(j+1)*unit_length]).to_csv(file_path, columns=None, index=None, header=None)

        # Get prediction
        pred = model(loader.x, loader.edge_index, loader.batch)
        pred = pred.cpu().detach().numpy()

        # Save prediction
        for j in range(batch_size):
            file_path = f'../../res/IEEE_Case1354/model_evaluation/agg_PG/pred/pred_{i*batch_size+j+1}.csv'
            pd.DataFrame(pred[j*unit_length:(j+1)*unit_length]).to_csv(file_path, columns=None, index=None, header=None)